# Build a Deck of Cards

In [27]:
#Function to build a deck of cards
def GenerateDeck():
    Deck = []
    Suits = ['Hearts', 'Spades', 'Clovers', 'Diamonds']
    Ranks = ['Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King','Ace']
    Suits_Ranks = []
    
    #Combine a list of all suits and values ex: 'Ace of Diamonds'
    for i in range(len(Suits)):
        for x in range(len(Ranks)):
            Suits_Ranks.append(Ranks[x] + ' of ' + Suits[i])
        
    BlackJack_Values = [2,3,4,5,6,7,8,9,10,10,10,10,11]
    BlackJack_Values_Final = []

    #Build a list of all possible values associated with each card
    for x in range(0,4):
        for i in range(len(BlackJack_Values)):
            BlackJack_Values_Final.append(BlackJack_Values[i])
            
    #Create cards as tuples. Each tuple contains two items. The suit/rank and value. All the cards are added to a list named Deck.    
    for i in range(len(BlackJack_Values_Final)):
        Card = (Suits_Ranks[i], BlackJack_Values_Final[i])
        Deck.append(Card) 
    return Deck

    




# Functions for Player/Dealer to Hit

In [28]:
#Function to add a card to the players hand. The running total value is updated based on the card drawn
#The counted number of aces in the players hand is updated as well
def Player_Hit(Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck):
    Player_Hand.append(Deck.pop())
    PlayerSum += Player_Hand[len(Player_Hand)-1][1]
    if (Player_Hand[len(Player_Hand)-1][1] == 11):
        AcesCount += 1
        Hard_Soft_Flag = 'Soft'
    return Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck

In [29]:
#Function to add a card to the dealers hand. The running total value is updated based on the card drawn
#The counted number of aces in the dealers hand is updated as well
def Dealer_Hit(Dealer_Hand, DealerSum, DealerAcesCount, Deck):
    Dealer_Hand.append(Deck.pop())
    DealerSum += Dealer_Hand[len(Dealer_Hand)-1][1]
    if (Dealer_Hand[len(Dealer_Hand)-1][1] == 11):
        DealerAcesCount += 1
    return Dealer_Hand, DealerSum, DealerAcesCount, Deck

# Player Perfect Strategy Logic

In [30]:
#Function that calculates what the player will do based on what cards they have and what card the dealer is showing
#Strategy is based on the left most table found at https://wizardofodds.com/games/blackjack/strategy/4-decks/
#I did not allow doubling or splits for this simulation
#Uses recurrsion to draw cards until a final player value is returned

def PerfectStrategy(Hard_Soft_Flag, PlayerSum, Player_Hand, AcesCount, DealerSum, Deck):
    Sum = PlayerSum
    Hand = Player_Hand
    Aces = AcesCount
    DeckCopy = Deck
    Flag = Hard_Soft_Flag
    
    #If the players hand does not contain an Ace the hand is considered 'Hard'. 
    #Logic therefore corresponds to the top section of the odds chart found via link at the top
    #Hand can become 'Soft' if an Ace is ever drawn
    if (Flag == 'Hard'):
        if ((Sum == 21) and (len(Hand) == 2)):
            #print('Player BlackJack \n')
            return 'BlackJack'
        elif (Sum > 21):
            #print('Player Bust \n')
            return 'Bust'
        elif (Sum <= 11):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + '\n')
            return PerfectStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)    
        elif (Sum >= 17):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        elif ((DealerSum <= 6) and (Sum >= 13)):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        elif ((DealerSum >= 7) and (Sum <= 16)):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + '\n')
            return PerfectStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)
        elif ((Sum == 12) and (DealerSum <= 3)):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + '\n')
            return PerfectStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)
        elif ((Sum == 12) and ((DealerSum <= 6) and (DealerSum >= 4))):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        
    #If players hand includes an ace the hand is considered 'Soft' and the middle section of the logic chart is used.
    if (Flag == 'Soft'):
        if ((Sum == 21) and (len(Hand) == 2)):
            #print('Player BlackJack \n')
            return 'BlackJack'
        #If a player busts via counting an ace as 11 then adjust the hand to count that ace as a one and retry the logic
        elif ((Sum > 21) and (Aces > 0)):
            Sum = Sum - 10
            Aces = Aces - 1
            if (Aces == 0):
                Flag = 'Hard'
                #print('New Player Sum = ' + str(Sum) + ' AcescCount is '+ str(Aces)+ 'Flag is ' + Flag +' \n')
            return PerfectStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)    
        elif ((Sum > 21) and (Aces == 0)):
            #print('Player Bust \n')
            return 'Bust'
        elif (Sum >= 19):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        elif (Sum <= 17):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + ' AcescCount is '+ str(Aces)+ ' \n')
            return PerfectStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy) 
        elif ((Sum == 18) and (DealerSum <= 8)):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        elif ((Sum == 18) and (DealerSum >= 9)):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + ' AcescCount is '+ str(Aces)+ ' \n')
            return PerfectStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)
            
            
        
            
        
        
        

# Dealer Strategy Logic

In [31]:
#Function that calculates what the dealer will do based on cards drawn
#Uses recurrsion to draw cards until a final player value is returned
def DealerStrategy(Dealer_Hand, DealerSum, DealerAcesCount, Deck):
    Aces = DealerAcesCount
    Sum = DealerSum
    Hand = Dealer_Hand
    DeckCopy = Deck
    if ((Sum == 21) and (len(Hand) == 2)):
        #print('Dealer BlackJack \n')
        return 'Dealer BlackJack'
    elif (Sum >= 17 and Sum <=21):
        
        #print('Final Dealer Total is: '+ str(Sum) + '\n')
        return Sum
    elif (Sum <= 16):
        Hand, Sum, Aces, DeckCopy = Dealer_Hit(Hand, Sum, Aces, DeckCopy)
        #print('Dealer Hit! New Dealer Sum = ' + str(Sum) + ' AcesCount is '+ str(Aces)+ ' \n')
        return DealerStrategy(Hand, Sum, Aces, DeckCopy)
    elif (Sum > 21 and Aces > 0):
        
        Sum = Sum - 10
        Aces = Aces - 1
        #print('Dealer has Ace and busted. New Dealer sum is ' + str(Sum) + ' New Aces Count is ' + str(Aces) + ' \n')
        return DealerStrategy(Hand, Sum, Aces, DeckCopy)
    elif (Sum > 21 and Aces == 0):
        #print('Dealer Bust \n')
        return 'Dealer Bust'
        
    

# Play One Hand of BlackJack Using the Perfect Strategy

In [32]:
import random


#Function to run through a single game of blackjack using the perfect strategy
#Returns the amount gained/lost based on the bet value input (default $1 bet)
def PlayOneHandPerfect(bet = 1):
    Deck = GenerateDeck()
    random.shuffle(Deck)
    Player_Hand = []
    Dealer_Hand = []
    PlayerSum = 0
    DealerSum = 0
    AcesCount = 0
    DealerAcesCount = 0
    
    #Default to hard. Change to Soft if an Ace is ever drawn
    Hard_Soft_Flag = 'Hard'
    
    #Initially draw two cards for the player and one face up card for the dealer
    Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck = Player_Hit(Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck)
    Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck = Player_Hit(Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck)
    Dealer_Hand, DealerSum, DealerAcesCount, Deck = Dealer_Hit(Dealer_Hand, DealerSum, DealerAcesCount, Deck)
    
    #print('Initial Player Total: ' + str(PlayerSum) + ' Flag is ' + Hard_Soft_Flag + ' \n')
    #print('Initial Dealer Total: ' + str(DealerSum) + '\n')
    
    #Calculate the final player and dealer values
    FinalPlayerTotal = PerfectStrategy(Hard_Soft_Flag, PlayerSum, Player_Hand, AcesCount, DealerSum, Deck)
    FinalDealerTotal = DealerStrategy(Dealer_Hand, DealerSum, DealerAcesCount, Deck)
    
    
    #Return the gain/loss amount based on comparing the final player and dealer values
    if (FinalPlayerTotal == 'Bust'):
        #print('Player made: ' + str(-bet) + '\n')
        return -bet
    elif (FinalPlayerTotal == 'BlackJack' and FinalDealerTotal == 'Dealer BlackJack'):
        #print('Player made: ' + str(0) + '\n')
        return 0
    elif  (FinalPlayerTotal == 'BlackJack'):
        #print('Player made: ' + str(bet*1.5) + '\n')
        return bet * 1.5
    elif (FinalDealerTotal == 'Dealer BlackJack'):
        #print('Player made: ' + str(-bet) + '\n')
        return -bet
    elif (FinalDealerTotal == 'Dealer Bust'):
        #print('Player made: ' + str(bet) + '\n')
        return bet
    elif (FinalPlayerTotal == FinalDealerTotal):
        #print('Player made: ' + str(0) + '\n')
        return 0
    elif (FinalPlayerTotal > FinalDealerTotal):
        #print('Player made: ' + str(bet) + '\n')
        return bet
    elif (FinalDealerTotal > FinalPlayerTotal):
        #print('Player made: ' + str(-bet) + '\n')
        return -bet


# Simulate a Large Number of Games Using the Perfect Strategy in Order to Calculate House Edge Percentage

In [46]:
#Function to play a large number of blackjack games using the perfect strategy
#Calculates and prints the total dollars gained/lossed and the estimated house edge percentage
def BlackJackSimPerfect(Hands = 1000000):
    Profit_Loss = 0
    for i in range(Hands):
        Return = PlayOneHandPerfect()
        #print ('***')
        Profit_Loss += Return
    House_Percentage = -Profit_Loss * 100 / Hands
    print('Total Profit//Loss Based on Playing ' + str(Hands) + ' Games: ' + str(Profit_Loss) + '\n')
    print('House Edge Percentage Based on Playing ' + str(Hands) + ' Games: ' + str(House_Percentage) + '% \n')
        

In [50]:
BlackJackSimPerfect()

Total Profit//Loss Based on Playing 1000000 Games: -20194.0

House Edge Percentage Based on Playing 1000000 Games: 2.0194% 



2.02% for the house edge seems to be a reasonable value. With all rules included the house edge is normally ~0.5%. No splits adds ~0.5% and no doubling adds ~1.4%. 

# Adjusted Strategy Based on How a Casual Player Might Play

In [41]:
#Copy of the perfect strategy from before with some adjustments
#Logic is based on how a casual tourist might play
#The dealer face up card is ignored. 
#Hard hands always stay at 16 or above and hit at 15 or below
#Soft hands always stay at 19 or above and hit at 18 or below


def TouristStrategy(Hard_Soft_Flag, PlayerSum, Player_Hand, AcesCount, DealerSum, Deck):
    Sum = PlayerSum
    Hand = Player_Hand
    Aces = AcesCount
    DeckCopy = Deck
    Flag = Hard_Soft_Flag
    
    #If the players hand does not contain an Ace the hand is considered 'Hard'. 
    #Logic therefore corresponds to the top section of the odds chart found via link at the top
    #Hand can become 'Soft' if an Ace is ever drawn
    if (Flag == 'Hard'):
        if ((Sum == 21) and (len(Hand) == 2)):
            #print('Player BlackJack \n')
            return 'BlackJack'
        elif (Sum > 21):
            #print('Player Bust \n')
            return 'Bust'
        elif (Sum <= 15):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + '\n')
            return TouristStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)    
        elif (Sum >= 16):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        
        
    #If players hand includes an ace the hand is considered 'Soft' and the middle section of the logic chart is used.
    if (Flag == 'Soft'):
        if ((Sum == 21) and (len(Hand) == 2)):
            #print('Player BlackJack \n')
            return 'BlackJack'
        #If a player busts via counting an ace as 11 then adjust the hand to count that ace as a one and retry the logic
        elif ((Sum > 21) and (Aces > 0)):
            Sum = Sum - 10
            Aces = Aces - 1
            if (Aces == 0):
                Flag = 'Hard'
                #print('New Player Sum = ' + str(Sum) + ' AcescCount is '+ str(Aces)+ 'Flag is ' + Flag +' \n')
            return TouristStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy)    
        elif ((Sum > 21) and (Aces == 0)):
            #print('Player Bust \n')
            return 'Bust'
        elif (Sum >= 19):
            #print ('Final Player Total is '+ str(Sum) + '\n')
            return Sum
        elif (Sum <= 18):
            Flag, Hand, Sum, Aces, DeckCopy = Player_Hit(Flag, Hand, Sum, Aces, DeckCopy)
            #print('Hit! New Player Sum = ' + str(Sum) + ' AcescCount is '+ str(Aces)+ ' \n')
            return TouristStrategy(Flag, Sum, Hand, Aces, DealerSum, DeckCopy) 
        
            

# Play One Hand of BlackJack Using the Tourist Strategy

In [42]:
#Function to run through a single game of blackjack using the tourist strategy
#Returns the amount gained/lost based on the bet value input (default $1 bet)
def PlayOneHandTourist(bet = 1):
    Deck = GenerateDeck()
    random.shuffle(Deck)
    Player_Hand = []
    Dealer_Hand = []
    PlayerSum = 0
    DealerSum = 0
    AcesCount = 0
    DealerAcesCount = 0
    
    #Default to hard. Change to Soft if an Ace is ever drawn
    Hard_Soft_Flag = 'Hard'
    
    #Initially draw two cards for the player and one face up card for the dealer
    Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck = Player_Hit(Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck)
    Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck = Player_Hit(Hard_Soft_Flag, Player_Hand, PlayerSum, AcesCount, Deck)
    Dealer_Hand, DealerSum, DealerAcesCount, Deck = Dealer_Hit(Dealer_Hand, DealerSum, DealerAcesCount, Deck)
    
    #print('Initial Player Total: ' + str(PlayerSum) + ' Flag is ' + Hard_Soft_Flag + ' \n')
    #print('Initial Dealer Total: ' + str(DealerSum) + '\n')
    
    #Calculate the final player and dealer values
    FinalPlayerTotal = TouristStrategy(Hard_Soft_Flag, PlayerSum, Player_Hand, AcesCount, DealerSum, Deck)
    FinalDealerTotal = DealerStrategy(Dealer_Hand, DealerSum, DealerAcesCount, Deck)
    
    
    #Return the gain/loss amount based on comparing the final player and dealer values
    if (FinalPlayerTotal == 'Bust'):
        #print('Player made: ' + str(-bet) + '\n')
        return -bet
    elif (FinalPlayerTotal == 'BlackJack' and FinalDealerTotal == 'Dealer BlackJack'):
        #print('Player made: ' + str(0) + '\n')
        return 0
    elif  (FinalPlayerTotal == 'BlackJack'):
        #print('Player made: ' + str(bet*1.5) + '\n')
        return bet * 1.5
    elif (FinalDealerTotal == 'Dealer BlackJack'):
        #print('Player made: ' + str(-bet) + '\n')
        return -bet
    elif (FinalDealerTotal == 'Dealer Bust'):
        #print('Player made: ' + str(bet) + '\n')
        return bet
    elif (FinalPlayerTotal == FinalDealerTotal):
        #print('Player made: ' + str(0) + '\n')
        return 0
    elif (FinalPlayerTotal > FinalDealerTotal):
        #print('Player made: ' + str(bet) + '\n')
        return bet
    elif (FinalDealerTotal > FinalPlayerTotal):
        #print('Player made: ' + str(-bet) + '\n')
        return -bet

# Simulate a Large Number of Games Using the Tourist Strategy in Order to Calculate House Edge Percentage

In [48]:
#Function to play a large number of blackjack games using the tourist strategy
#Calculates and prints the total dollars gained/lossed and the estimated house edge percentage
def BlackJackSimTourist(Hands = 1000000):
    Profit_Loss = 0
    for i in range(Hands):
        Return = PlayOneHandTourist()# Simulate a Large Number of Games Using the Perfect Strategy in Order to Calculate House Edge Percentage()
        #print ('***')
        Profit_Loss += Return
    House_Percentage = -Profit_Loss * 100 / Hands
    print('Total Profit//Loss Based on Playing ' + str(Hands) + ' Games: ' + str(Profit_Loss) + '\n')
    print('House Edge Percentage Based on Playing ' + str(Hands) + ' Games: ' + str(House_Percentage) + '% \n')

In [49]:
BlackJackSimTourist()

Total Profit//Loss Based on Playing 1000000 Games: -46855.0

House Edge Percentage Based on Playing 1000000 Games: 4.6855% 



Based on my adjustments to the logic the house edge increased ~2.5% based on ignoring the dealer face up card.